In [ ]:
import pandas as pd
# Load the file layout
layout = pd.read_excel("../Data/mortgage_data/file_layout.xlsx", sheet_name=None)

## Column name extraction from Freddie Mac documentation

In [ ]:
# sheet_names = layout.keys()
# Extract column names and data types for both origination and performance datasets
orig_layout = layout['Origination Data File']
perf_layout = layout['Monthly Performance Data File']

# Extract column names and data types
orig_column_names = orig_layout['ATTRIBUTE NAME'].tolist()
orig_data_types = orig_layout['DATA TYPE & FORMAT'].tolist()
perf_column_names = perf_layout['ATTRIBUTE NAME'].tolist()
perf_data_types = perf_layout['DATA TYPE & FORMAT'].tolist()

In [ ]:
# # Load origination sample data for 2022
# orig_sample = pd.read_csv("../Data/mortgage_data/sample_2022/sample_orig_2022.txt", sep="|", header=None, names=orig_column_names, low_memory=False)
# # Load performance sample data for 2022
# perf_sample = pd.read_csv("../Data/mortgage_data/sample_2022/sample_svcg_2022.txt", sep="|", header=None, names=perf_column_names, low_memory=False)
# # Display first few rows of both datasets
# orig_sample.head()

## Load data per year

In [ ]:
def drop_cols_and_NAN(data, cols_to_drop):
    #first drop all columns that only have NaN values
    data = data.dropna(axis=1, how='all')
    #drop cols_to_drop
    data = data.drop(cols_to_drop, axis=1)
    return data

NameError: name 'perf_cols_drop' is not defined

In [ ]:
def load_yearly_data(year, base_dir="../Data/mortgage_data"):
    """
    Load and format the origination and performance datasets for a given year, considering the folder structure.
    
    Parameters:
    - year: The year for which to load the data.
    - base_dir: The base directory where the datasets are stored.
    
    Returns:
    - orig_data: Formatted origination dataset for the given year.
    - perf_data: Formatted performance dataset for the given year.
    """


    # Construct file paths considering the "sample_YYYY" folder structure
    orig_file_path = f"{base_dir}/sample_{year}/sample_orig_{year}.txt"
    perf_file_path = f"{base_dir}/sample_{year}/sample_svcg_{year}.txt"
    
    # Load origination data
    orig_data = pd.read_csv(orig_file_path, sep="|", header=None, names=orig_column_names, low_memory=False)
    # Load performance data
    perf_data = pd.read_csv(perf_file_path, sep="|", header=None, names=perf_column_names, low_memory=False)
    
    perf_cols_drop = []#select columns you want to drop
    orig_cols_drop = []#select columns you want to drop
    orig_data = drop_cols_and_NAN(orig_data, orig_cols_drop)
    perf_data = drop_cols_and_NAN(perf_data, perf_cols_drop)

    return orig_data, perf_data


## Load all data at once into dictionary

In [6]:
def load_all_datasets(start_year=1999, end_year=2022, base_dir="../Data/mortgage_data/"):
    """
    Load all origination and performance datasets for a given range of years.
    
    Parameters:
    - start_year: The starting year (inclusive) for which to load the data.
    - end_year: The ending year (inclusive) for which to load the data.
    - base_dir: The base directory where the datasets are stored.
    
    Returns:
    - datasets: Dictionary containing formatted origination and performance datasets for the given range of years.
    """
    
    datasets = {}
    
    for year in range(start_year, end_year + 1):
        orig_data, perf_data = load_yearly_data(year, base_dir=base_dir)
        
        datasets[f"orig_{year}"] = orig_data
        datasets[f"perf_{year}"] = perf_data
    
    return datasets

# For demonstration purposes, we'll load only the 2022 sample data
# To load all years' data, you would simply call load_all_datasets() without the year range
datasets_demo = load_all_datasets(start_year=1999, end_year=2022)
datasets_demo.keys()  # Display the keys of the dictionary

dict_keys(['orig_1999', 'perf_1999', 'orig_2000', 'perf_2000', 'orig_2001', 'perf_2001', 'orig_2002', 'perf_2002', 'orig_2003', 'perf_2003', 'orig_2004', 'perf_2004', 'orig_2005', 'perf_2005', 'orig_2006', 'perf_2006', 'orig_2007', 'perf_2007', 'orig_2008', 'perf_2008', 'orig_2009', 'perf_2009', 'orig_2010', 'perf_2010', 'orig_2011', 'perf_2011', 'orig_2012', 'perf_2012', 'orig_2013', 'perf_2013', 'orig_2014', 'perf_2014', 'orig_2015', 'perf_2015', 'orig_2016', 'perf_2016', 'orig_2017', 'perf_2017', 'orig_2018', 'perf_2018', 'orig_2019', 'perf_2019', 'orig_2020', 'perf_2020', 'orig_2021', 'perf_2021', 'orig_2022', 'perf_2022'])

In [10]:
def merge_orig_with_perf(orig_data, perf_data):
    merged_data = pd.merge(perf_data, orig_data, on="Loan Sequence Number", how="left", inplace = True)
    return merged_data

In [12]:
fm_2022 = merge_orig_with_perf(datasets_demo['orig_2022'], datasets_demo['perf_2022'])

In [16]:
#which column only contain NaN values in fm_2022
fm_2022.isnull().all()
#drop columns that only contain NaN values
fm_2022.dropna(axis=1,how='all',inplace=True)

,Loan Sequence Number,Monthly Reporting Period,Current Actual UPB,Current Loan Delinquency Status,Loan Age,Remaining Months to Legal Maturity,Defect Settlement Date,Modification Flag,Zero Balance Code,Zero Balance Effective Date,...,Loan Purpose,Original Loan Term,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Program Indicator,Property Valuation Method,Interest Only (I/O) Indicator,Mortgage Insurance Cancellation Indicator
0,F22Q10000012,202202,95000.0,0,0,180,NaN,NaN,NaN,NaN,...,N,180,1,Other sellers,Other servicers,NaN,9,2,N,7
1,F22Q10000012,202203,94000.0,0,1,179,NaN,NaN,NaN,NaN,...,N,180,1,Other sellers,Other servicers,NaN,9,2,N,7
2,F22Q10000012,202204,94000.0,0,2,178,NaN,NaN,NaN,NaN,...,N,180,1,Other sellers,Other servicers,NaN,9,2,N,7
3,F22Q10000012,202205,93000.0,0,3,177,NaN,NaN,NaN,NaN,...,N,180,1,Other sellers,Other servicers,NaN,9,2,N,7
4,F22Q10000012,202206,93000.0,0,4,176,NaN,NaN,NaN,NaN,...,N,180,1,Other sellers,Other servicers,NaN,9,2,N,7


In [17]:
fm_2022.describe()

,Monthly Reporting Period,Current Actual UPB,Current Loan Delinquency Status,Loan Age,Remaining Months to Legal Maturity,Defect Settlement Date,Zero Balance Code,Zero Balance Effective Date,Current Interest Rate,Current Deferred UPB,...,Number of Units,Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,Original UPB,Original Loan-to-Value (LTV),Original Interest Rate,Postal Code,Original Loan Term,Number of Borrowers,Property Valuation Method
count,419440.000000,4.194400e+05,419440.000000,419440.000000,419440.000000,89.000000,1103.000000,1103.000000,419440.000000,419440.000000,...,419440.000000,419440.000000,419440.000000,4.194400e+05,419440.000000,419440.000000,419440.000000,419440.000000,419440.000000,419440.000000
mean,202241.501476,2.925162e+05,0.014004,4.453369,331.845327,202238.797753,8.665458,202249.543971,4.664095,0.822730,...,1.033547,73.291009,36.752868,2.977420e+05,73.183764,4.664095,52234.633082,336.299051,1.446281,1.840211
std,44.616326,1.578596e+05,0.217235,3.355279,59.637687,43.386000,25.885647,46.352567,1.180446,90.033605,...,0.237701,18.342914,15.328947,1.586295e+05,18.342691,1.180446,28577.100570,59.311845,0.527007,0.385289
min,202202.000000,0.000000e+00,0.000000,0.000000,83.000000,202205.000000,1.000000,202202.000000,1.750000,0.000000,...,1.000000,6.000000,1.000000,1.600000e+04,6.000000,1.750000,700.000000,96.000000,1.000000,1.000000
25%,202208.000000,1.730000e+05,0.000000,2.000000,351.000000,202209.000000,1.000000,202209.000000,3.750000,0.000000,...,1.000000,62.000000,30.000000,1.760000e+05,62.000000,3.750000,30000.000000,360.000000,1.000000,2.000000
50%,202211.000000,2.620000e+05,0.000000,4.000000,355.000000,202211.000000,1.000000,202212.000000,4.625000,0.000000,...,1.000000,78.000000,38.000000,2.670000e+05,77.000000,4.625000,48100.000000,360.000000,1.000000,2.000000
75%,202301.000000,3.836361e+05,0.000000,7.000000,358.000000,202302.000000,1.000000,202302.000000,5.500000,0.000000,...,1.000000,90.000000,44.000000,3.900000e+05,90.000000,5.500000,78500.000000,360.000000,2.000000,2.000000
max,202303.000000,1.483000e+06,13.000000,14.000000,480.000000,202303.000000,96.000000,202303.000000,8.125000,22766.580000,...,4.000000,105.000000,999.000000,1.485000e+06,97.000000,8.125000,99900.000000,360.000000,5.000000,9.000000


## Function to merge macro data to performance

In [ ]:
def merge_macro_data(perf_data, macro_data, perf_date_col, macro_date_col):
    """
    Merge macroeconomic data with the performance dataset based on the date.
    
    Parameters:
    - perf_data: The performance dataset.
    - macro_data: The macroeconomic dataset.
    - perf_date_col: The date column name in the performance dataset.
    - macro_date_col: The date column name in the macroeconomic dataset.
    
    Returns:
    - Merged dataset.
    """
    
    # Ensure the date columns are in a monthly format
    perf_data[perf_date_col] = pd.to_datetime(perf_data[perf_date_col]).dt.to_period('M')
    macro_data[macro_date_col] = pd.to_datetime(macro_data[macro_date_col]).dt.to_period('M')
    
    # Merge datasets based on the date
    merged_data = pd.merge(perf_data, macro_data, left_on=perf_date_col, right_on=macro_date_col, how='left')
    return merged_data

# Sample usage of the functions can be provided if datasets are available.
# For now, these functions are generic and can be adapted to actual data.


## Function to merge zip data to performance

In [ ]:
def merge_zip_data(perf_data, zip_data, perf_zip_col, zip_data_col):
    """
    Merge external data with the performance dataset based on the 3zip code.
    
    Parameters:
    - perf_data: The performance dataset.
    - zip_data: The external dataset with 3zip level information.
    - perf_zip_col: The zip column name in the performance dataset (might be 5-digit zip).
    - zip_data_col: The 3zip column name in the external dataset.
    
    Returns:
    - Merged dataset.
    """
    # Convert 5-digit zip code to 3zip format
    perf_data['3zip'] = perf_data[perf_zip_col].astype(str).str[:3]
    zip_data['3zip'] = zip_data[zip_data_col].astype(str).str[:3]
    
    # Merge datasets based on the 3zip code
    merged_data = pd.merge(perf_data, zip_data, left_on='3zip', right_on='3zip', how='left')
    
    return merged_data